# Libraries

In [1]:
# Import libraries

from os import listdir
import string
import pandas as pd
import numpy as np  
import pandas as pd 
import re           
from bs4 import BeautifulSoup 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from attention import AttentionLayer
import warnings
import re, string, unicodedata
import nltk
import pickle
import inflect
from bs4 import BeautifulSoup
import tensorflow as tf
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 

pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [2]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


# Load data

In [3]:
x_test = np.load("../../../final_data/x_test.npy")
y_test = np.load("../../../final_data/y_test.npy")

In [4]:
#!pip install pickle5
import pickle

In [5]:
def open_test_data_y():
    return open('../../../tokenizers_vars/y_tokenizer.pickle', 'rb')

with open_test_data_y() as f:
    y_tokenizer = pickle.load(f) 

def open_test_data_x():
    return open('../../../tokenizers_vars/x_tokenizer.pickle', 'rb')

with open_test_data_x() as f:
    x_tokenizer = pickle.load(f) 

def open_vars():
    return open('../../../tokenizers_vars/vars.pkl', 'rb')

with open_vars() as f:  # Python 3: open(..., 'rb')
    x_voc, y_voc = pickle.load(f)

In [6]:
max_text_len=300
max_summary_len=12

# Inference

In [18]:
model = tf.keras.models.load_model('saved_model')

In [19]:
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

In [21]:

latent_dim = 300
embedding_dim=100


# Encode the input sequence to get the feature vector
encoder_inputs = model.input[0]   # input_1
encoder_outputs, forward_h, forward_c, backward_h, backward_c = model.layers[3].output 
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])


# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_inputs = model.input[1]
decoder_state_input_h = Input(shape=(latent_dim*2,), name='dec_st_in_h')
decoder_state_input_c = Input(shape=(latent_dim*2,), name='dec_st_in_c')
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim*2))

# Get the embeddings of the decoder sequence
dec_emb_layer = model.layers[4]
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_lstm = model.layers[7]
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_layer = model.layers[8]
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_dense = model.layers[10]
decoder_outputs2 = decoder_dense(decoder_inf_concat) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [25]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['starttoken']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='endtoken'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'endtoken'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [26]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['starttoken']) and i!=target_word_index['endtoken']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [27]:
for i in range(0,1):
    print("Review:",seq2text(x_test[i]))
    print("Original summary:",seq2summary(y_test[i]))
    print("Predicted summary:",decode_sequence(x_test[i].reshape(1,max_text_len)))
    print("\n")

Review: movie cost million make winner obvious mark talks direct shining prequel apes strong start two weeks however maintain stand one stronger films summer friday seemed purge would take spoils title well movie earned million friday night showings compared apes lackluster million percent drop opening night week apes rallied big time saturday earning million purge million percent drop opening night writing wall apes slot went planes fire rescue studios sequel last year family comedy adventurous airplanes spun pixar cars universe million mil less first planes pretty bad stall family movie middle summer low friday gross million seemed like may simply reflection many adults turned opening night fare fire rescue much better family friendly matinee hours saturday earning million cinemascore rating turn enjoyed saw said cameron diaz jason segel sex tape nabbed dismal c plus cinemascore opened fourth place million diaz comedy hot streak lately raunchy slapstick couple trying prevent friends 

In [48]:
original_text = []
original_summary = []
created_summary = []

for i in range(0,100):
    original_text.append(seq2text(x_test[i]))
    original_summary.append(seq2summary(y_test[i]))
    created_summary.append(decode_sequence(x_test[i].reshape(1,max_text_len)))

In [49]:
results = pd.DataFrame()
results["Original_text"] = original_text
results["Original_summary"] = original_summary
results["Created_summary"] = created_summary

In [50]:
results.to_csv("results_predictions_mono_10.csv")

In [51]:
results=pd.read_csv("results_predictions_mono_10.csv")

In [52]:
reference_sentences = results["Original_summary"].to_list()
summary_sentences = results["Created_summary"].to_list()

# Evaluation

In [53]:
from rouge import rouge_n_sentence_level
from rouge import rouge_l_sentence_level
from rouge import rouge_n_summary_level
from rouge import rouge_l_summary_level
from rouge import rouge_w_sentence_level
from rouge import rouge_w_summary_level

list_rouge = []
list_recall = []
list_precision = []

for i in range(0, len(reference_sentences)):
    reference_sentence = reference_sentences[i].split()
    summary_sentence = summary_sentences[i].split()
    
    # Calculate ROUGE-2.
    recall, precision, rouge = rouge_n_sentence_level(summary_sentence, reference_sentence, 2)
    #print('ROUGE-2-R', recall)
    #print('ROUGE-2-P', precision)
    #print('ROUGE-2-F', rouge)

    list_rouge.append(rouge)
    list_recall.append(recall)
    list_precision.append(precision)

import statistics


mean_rouge = statistics.mean(list_rouge)  
print(mean_rouge)

0.013205128205128204
